In [1]:
import pandas as pd
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
import re
import time

import matplotlib.pyplot as plt
%matplotlib inline

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 960 (0000:01:00.0)


In [2]:
if os.path.exists('business.json'):
    pass
else:
    !wget https://s3.amazonaws.com/cloudfinalprojectsubmission/business.json

if os.path.exists('review.json'):
    pass
else:
    !wget https://s3.amazonaws.com/cloudfinalprojectsubmission/review.json

In [3]:
2+3

5

In [4]:
chunk_size = 100000

In [5]:
usa_business_idx = []
business_reader = pd.read_table('business.json', header=None, chunksize=chunk_size)
categories = pd.DataFrame()

try:
    for i, chunks in enumerate(business_reader):

        start_index = chunk_size * i

        for index in range(start_index, start_index + len(chunks)):

            data_json = pd.read_json(chunks[0][index], typ='series')
            if data_json['city'] in ['Pittsburgh', 'Charlotte', 'Urbana-Champaign',
                                     'Phoenix', 'Las Vegas', 'Madison']:
                usa_business_idx.append(data_json['business_id'])
                categories = categories.append(data_json[['business_id', 'categories']], ignore_index=True)
except:
    print(i)

In [6]:
categories = categories.drop_duplicates('business_id')
categories = categories.set_index('business_id')

In [7]:
categories.shape

(11487, 1)

In [8]:
review_reader = pd.read_table('review.json', header=None, chunksize=chunk_size)
df = pd.DataFrame()


try:
    for i, chunks in enumerate(review_reader):

        start_index = chunk_size * i
        for index in range(start_index, start_index + len(chunks)):
                data_json = pd.read_json(chunks[0][index], typ='series')
                bus_id = data_json['business_id']
                if (bus_id in usa_business_idx) and (u'Restaurants' in categories.loc[bus_id].categories):
                    df = df.append(data_json[['text', 'stars']], ignore_index=True)
                    if data_json['stars'] < 4.:
                        df = df.append(data_json[['text', 'stars']], ignore_index=True)
except:
    pass

In [9]:
df.shape

(26227, 2)

In [10]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
import scipy.sparse as sp
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
#from keras.regularizers import l2, activity_l2
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.optimizers import Nadam
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
#import word2vecReader
import word2vec
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer
from sklearn.ensemble import RandomForestClassifier as RandForest
from sklearn.svm import SVC
from nltk.tokenize import RegexpTokenizer

/home/svetlana/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [11]:
label_encoder = LabelEncoder()

In [12]:
df = pd.DataFrame(df)

In [13]:
df['text']

0        Went for breakfast on 6/16/14. We received ver...
1        I love their stakes and i come here every week...
2        Perfect time with hubby, after work which is 4...
3        This location never disappoints!! Food is alwa...
4        Dude... it's Denny's. It's excellent drunk foo...
5        Dude... it's Denny's. It's excellent drunk foo...
6        Excellent food , good service too I have had s...
7        Chose Denny's for a very late (1:30pm) Sunday ...
8        I like the food at Denny's more than meals ser...
9        This is the best Denny's in the area. Friendly...
10       Got seated right away and the staff was very p...
11       Got seated right away and the staff was very p...
12       I'm sorry to give any stars...15 minutes for a...
13       I'm sorry to give any stars...15 minutes for a...
14       We decided on Sapporo after reading so-so revi...
15       I travel a lot and have eaten at a lot of Hiba...
16       The food was alright. Not good. Not bad. And c.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['stars'], test_size=0.33, random_state=42)

In [15]:
max_len = 140

def lowercase(string):
    return string[:max_len].lower()

X_train['letters'] = X_train.apply(lowercase).apply(list)
X_test['letters'] = X_test.apply(lowercase).apply(list)

In [16]:
import string
from sklearn.preprocessing import LabelBinarizer
lbin = LabelBinarizer()

alphabet = list(string.ascii_lowercase + string.digits + string.punctuation + ' ')
alphabet_len = len(alphabet)
lbin.fit(alphabet)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

### one-hot coding

In [17]:
def binarize(char_list):
    binarized = np.zeros((max_len, alphabet_len))
    char = lbin.transform(char_list)
    binarized[:char.shape[0], :] = char
    binarized = binarized.reshape((1, max_len, alphabet_len))
    return binarized

In [18]:
from keras.models import Sequential
from keras.layers import Activation, MaxPool1D, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, Convolution1D, Convolution2D
from keras.utils import np_utils

In [19]:
X_train = np.vstack(X_train['letters'].apply(binarize)).astype('float32')
y_train = y_train.astype('float32')

X_test = np.vstack(X_test['letters'].apply(binarize)).astype('float32')
y_test = y_test.astype('float32')

group_lb = LabelBinarizer()
y_train = group_lb.fit_transform(y_train).astype('float32')
y_test = group_lb.transform(y_test).astype('float32')

In [20]:
from keras import backend as K

def my_init(shape, dtype=None):
    return K.random_normal(shape, stddev=0.05, dtype=dtype)    

def conv_model(tweet_len, symbols_num):
    
    inputs = Input(shape=(tweet_len, symbols_num))
    l_conv1 = Convolution1D(filters=512, kernel_size=2, activation='relu', kernel_initializer=my_init)(inputs)
    l_pool1 = MaxPool1D(pool_size=3)(l_conv1)
    bn1 = BatchNormalization()(l_pool1)
   
    l_conv2 = Convolution1D(filters=64, kernel_size=2, activation='relu')(bn1)
    l_pool2 = MaxPool1D(pool_size=4)(l_conv2)
    
    #l_conv3 = Convolution1D(filters=128, kernel_size=3, activation='relu')(l_pool2)
    #l_pool3 = MaxPool1D(pool_size=4)(l_conv3)



    #l_conv3 = Convolution1D(filters=128, kernel_size=3, activation='relu')(l_pool2)
    
    #l_conv4 = Convolution1D(filters=128, kernel_size=3, activation='relu')(l_conv3)
    #l_pool4 = MaxPool1D(pool_size=3)(l_conv4)
    
    #l_conv5 = Convolution1D(filters=256, kernel_size=3, activation='relu')(l_conv4)
    #l_pool5 = MaxPool1D(pool_size=3)(l_conv5)
    
    
    fc_layer_1 = Dense(512, activation='relu')(l_pool2)
    drop1_out = Dropout(0.5)(fc_layer_1)    
    
    flat = Flatten()(drop1_out)
    fc_layer_2 = Dense(128, activation='relu')(flat)
    drop2_out = Dropout(0.5)(fc_layer_2)
    
    #fc_layer_3 = Dense(128, activation='relu')(drop2_out)
    #drop3_out = Dropout(0.5)(fc_layer_2)
    
    fc_layer_3 = Dense(128, activation='relu')(drop2_out)
    drop3_out = Dropout(0.5)(fc_layer_3)
    
    out = Dense(5,activation='softmax')(drop3_out)
    model = Model(inputs, out)
    
    #Compilation
    model.compile(loss='categorical_crossentropy', optimizer = 'adam' , metrics=['accuracy'])
    model.summary()
    return model

# Charecter level CNN

In [21]:
model = conv_model(tweet_len=max_len, symbols_num=alphabet_len)
model.optimizer.lr.set_value(0.0001)
nb_epoch = 50
for epoch_num in range(nb_epoch):
    fit = model.fit(X_train, y_train, batch_size=30, epochs=1, verbose=1, validation_data=(X_test, y_test),)
    res = model.predict(X_test)
    print("epoch: ", epoch_num)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 140, 69)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 139, 512)          71168     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 512)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 512)           2048      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 45, 64)            65600     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 11, 512)           33280     
__________

In [1]:
pred_list = []
nb_bag = 5
for i in range(nb_bag):
    #weights = [np.random.normal(scale=0.05 ,size=(1, max_len, alphabet_len)), np.random.normal(scale=0.05, size=)]    
    model = conv_model(tweet_len=max_len, symbols_num=alphabet_len)
    nb_epoch = 30
    for epoch_num in range(nb_epoch):
        fit = model.fit(X_train, y_train, batch_size=256, epochs=1, verbose=1, validation_data=(X_test, y_test),
                        class_weight=class_weight)
        res = model.predict(X_test)
        print("epoch: ", epoch_num)
        print(f1_score(group_lb.inverse_transform(y_test), np.argmax(res, axis=1), average='macro'))
    pred_list.append(res)

NameError: name 'conv_model' is not defined